In [1]:
import numpy as np
import os
from PIL import Image
import matplotlib.pyplot as plt 
%matplotlib inline
from torchvision.transforms import ToTensor, ToPILImage
import glob
from skimage.metrics import structural_similarity as skssim
from skimage.metrics import peak_signal_noise_ratio as skpsnr


# 图片读写

In [ ]:
im=Image.open('picture/shen.jpg')
print(im.format, im.size, im.mode)
imarray=np.array(im)
#img.show()
plt.imshow(im)
plt.axis('on') # 关掉坐标轴为 off
plt.title('image') # 图像题目
plt.show()
print(imarray.shape)

In [ ]:
img = Image.fromarray(imarray)
plt.imshow(img)

# 图片裁剪

In [ ]:
#手机截图剪切至1920*1080
indir = 'pili'
outdir = 'pl'
st = 266
for i in os.listdir(indir):
    img = Image.open(os.path.join(indir,i))
    imgnp = np.array(img)[:,st:st+1920,:]
    imgs  = Image.fromarray(imgnp)
    #imgs.save(os.path.join(outdir,i))

# 图片切片

In [ ]:
in_dir = 'E:\Code\Python\datas\RS\ITC_VD_Training_Testing_set\Testing\Image'
out_dir = 'E:\Code\Python\datas\RS\ITC_VD_Training_Testing_set\Testing\patch'
#in_dir = 'test'
#out_dir = 'test_p'

In [ ]:
for imgi in os.listdir(in_dir):
    img = Image.open(os.path.join(in_dir,imgi))
    imgt = ToTensor()(img)
    c,h,w = imgt.shape
    for i in range(int(h/600)):
        for j in range(int(w/600)):
            x = imgt[:,i*600:(i+1)*600,j*600:(j+1)*600]
            ToPILImage()(x).save(f'{out_dir}/{imgi[2:-4]}_{i}_{j}.png')

# 批量修改倍数大小

In [ ]:
def ResizeImage(indir, outdir, scale):
    if not os.path.exists(outdir):
        os.mkdir(outdir)
    hrs = os.listdir(indir)
    for hr in hrs:
        img = Image.open(os.path.join(indir,hr))
        lr = img.resize((int(img.size[0]/scale),int(img.size[1]/scale)),Image.BICUBIC) 
        outname = hr[:-4]+'x'+str(scale)+hr[-4:]
        lr.save(os.path.join(outdir,outname))

In [ ]:
ResizeImage(r'E:\Code\Python\datas\RS\ITCVD_patch\ITCVD_test_patch'r'E:\Code\Python\datas\RS\ITCVD_patch\ITCVD_test_patchx23,43

In [ ]:
ResizeImage(r'DIV2K_valid_HR',r'DIV2K_valid_LRx30',30)

# 单点像素值

In [ ]:
print(img.getpixel((50,50)))

# 区域提取

In [ ]:
box = im.copy() #直接复制图像
box = (50, 300, 600, 700)
region = im.crop(box)
plt.imshow(region)
plt.imshow(region.resize((128,72),Image.LANCZOS))

# 通道分离

In [ ]:
img=im
gray = img.convert('L')
r,g,b = img.split()                          #注意：gray和r，g，b不一样，gray是三者通过公式算出来的
img_merged = Image.merge('RGB', (r, g, b))

#plt.figure(figsize=(10,5)) #设置窗口大小
plt.suptitle('Multi_Image') # 图片名称

plt.subplot(2,3,1), plt.title('imge:oringe')
plt.imshow(img), plt.axis('off')

plt.subplot(2,3,2), plt.title('gray:weicaise')
plt.imshow(gray), plt.axis('off')

plt.subplot(2,3,3), plt.title('gray:true')# 这里必须加 cmap='gray' ,否则尽管原图像是灰度图（下图1），但是显示的是伪彩色图像（下图2）（如果不加的话）
plt.imshow(gray,cmap='gray'), plt.axis('off')

plt.subplot(2,3,4), plt.title('r')
plt.imshow(r,cmap='gray'), plt.axis('off')

plt.subplot(2,3,5), plt.title('g')
plt.imshow(g,cmap='gray'), plt.axis('off')

plt.subplot(2,3,6), plt.title('b')
plt.imshow(b,cmap='gray'), plt.axis('off')

#plt.show()


# 计算图片SSIM

In [2]:
from ssim import ssim

In [2]:
def image_self_psnr(img_path,scale=2):
    hr = Image.open(img_path)
    hrnp = np.array(hr)
    h,w,c = hrnp.shape
    lr = hr.resize((h//2,w//2))
    lrnp = np.array(lr)
    sr = lr.resize((h,w))
    srnp = np.array(sr)
    psnr = skpsnr(hrnp,srnp)


    plt.subplot(2,2,1)
    plt.imshow(hr)
    plt.show()
    return psnr

In [57]:
def image_class_psnr(root):
    total_psnr = 0
    all_images = os.listdir(root)
    for img in all_images:
        imgpath = os.path.join(root,img)
        psnr = image_self_psnr(imgpath)
        #print(psnr)
        total_psnr += psnr
    return total_psnr/len(all_images)

In [56]:
image_class_psnr(r"WHURS19-test\Airport")

32.08768509688502
30.016250966322637
34.056229496219636
32.70773107857282
34.10794676439004
39.393495919978854
30.144345933675073
29.46089294403796
31.702944431634368
29.139573547368386


32.281709617908476

In [58]:
image_class_psnr(r"WHURS19-test\Beach")

46.139038395649315

In [ ]:
image_class_psnr(r"WHURS19-test\River")

Error: Session cannot generate requests

In [ ]:
image_self_psnr(r"WHURS19-test\Beach\beach_41.jpg")

Error: Session cannot generate requests

In [ ]:
hr = Image.open(r"WHURS19-test\Beach\beach_41.jpg")
plt.subplot(2,2,1)
plt.imshow(hr)
#plt.show()

Error: Session cannot generate requests

In [6]:
def get_img_by_name(classname):
    dict = {}
    for imgpath in glob.glob(f"WHURS19-test/{classname}*"):
        img = Image.open(imgpath)
        dict[os.path.basename(imgpath)] = ToTensor()(img).unsqueeze(0)
    return dict

In [10]:
airports = get_img_by_name("airport")

PermissionError: [Errno 13] Permission denied: 'WHURS19-test\\Airport'

In [ ]:
beachs = get_img_by_name("beach")

Error: Session cannot generate requests

In [ ]:
 bridges = get_img_by_name("bridge")

In [ ]:
industrials = get_img_by_name("industrial")

In [ ]:
def compare(ca,cb): 
    for a in ca:
        for b in cb:
            calssim = ssim(ca[a], cb[b])
            print(f"{a} vs {b} ssim : {calssim}")
    print("")

In [ ]:
compare(airports,beachs)

In [ ]:
ssim(beachs['beach_41.jpg'],airports['airport_41.jpg'])

In [ ]:
compare(airports,bridges)

In [ ]:
 
def get_thumbnail(image, size=(1200, 750), greyscale=False):
    image = image.resize(size, Image.ANTIALIAS)
    if greyscale:
        image = image.convert('L')
    return image
 
 
def image_similarity_vectors_via_numpy(image1, image2):
    image1 = get_thumbnail(image1)
    image2 = get_thumbnail(image2)
    images = [image1, image2]
    vectors = []
    norms = []
    for image in images:
        vector = []
        for pixel_tuple in image.getdata():
            vector.append(average(pixel_tuple))
        vectors.append(vector)
        norms.append(linalg.norm(vector, 2))
    a, b = vectors
    a_norm, b_norm = norms
    res = dot(a / a_norm, b / b_norm)
    return res



In [ ]:
from numpy import average, linalg, dot

In [ ]:
image1 = Image.open(r'WHURS19-test\Desert_41.jpg')
image2 = Image.open(r'WHURS19-test\river_41.jpg')
cosin = image_similarity_vectors_via_numpy(image1, image2)

In [ ]:
print(cosin)